In [274]:
# Importing necessary libraries

import numpy as np
import pandas as pd
import os
import cmath

In [275]:
# Reading bus data into panda dataframes

def buses(file):
    df_buses = pd.read_csv(file, sep=";")
    V = df_buses.V.to_numpy()
    delta = df_buses.Angle.to_numpy()
    P = df_buses.P_gen.to_numpy()
    Q = df_buses.Q_gen.to_numpy()
    bus_type = df_buses.BusType.to_numpy()
    return V,delta,P,Q, bus_type


V,delta,P,Q, bus_type = buses('Busdata.csv')

print(V)
print(delta)
print(P)
print(Q)
print(bus_type)


# Creating additional vectors that will be used and updaed during the calculations
V_calc, delta_calc, P_calc, Q_calc, bus_type = buses('Busdata.csv')

for x in range(len(V_calc)):
    if (np.isnan(V_calc[x])):
        V_calc[x] = 1
    if(np.isnan(delta_calc[x])):
        delta_calc[x] = 0

[ 1. nan  1. nan nan]
[nan nan  0. nan nan]
[ 1.  -0.6  nan -0.6 -0.5]
[ nan -0.3  nan -0.2 -0.4]
[1 2 0 2 2]


In [276]:
# Function to create the Y-bus matrix
# shape needs to be specified per now. Fix this?
def Ybus(file, shape):
    df_impedances = pd.read_csv(file, sep=";")
    print(df_impedances.head())
    Z_values = np.zeros((shape,shape), dtype=complex)
    Y_bus = np.zeros((shape,shape), dtype=complex)

    for x in range(df_impedances.shape[0]):
        from_line = df_impedances['From_line'][x]
        to_line = df_impedances['To_line'][x]

        # Adding the diagonal elements to the Y-bus
        Y_bus[from_line-1][from_line-1] += 1/(df_impedances['R'][x] + df_impedances['X'][x]*1j)
        Y_bus[from_line-1][from_line-1] += 1/2*(df_impedances['Full_line_B'][x]*1j)
        Y_bus[to_line-1][to_line-1] += 1/(df_impedances['R'][x] + df_impedances['X'][x]*1j)
        Y_bus[to_line-1][to_line-1] += 1/2*(df_impedances['Full_line_B'][x]*1j)

        # Z values for off diagonal elements
        Z_values[from_line-1][to_line-1] = df_impedances['R'][x] + df_impedances['X'][x]*1j
        Z_values[to_line-1][from_line-1] = df_impedances['R'][x] + df_impedances['X'][x]*1j
    # Adding off diagonal elements
    for i in range(shape):
        for j in range(shape):
            if(Z_values[i][j] != 0. +0.j):
                if(i != j):
                    Y_bus[i][j] = - 1/Z_values[i][j]
            else:
                if(i != j):
                    Y_bus[i][j] = Z_values[i][j]
    return Y_bus

In [277]:
Ybus = Ybus('impedances.csv', 5)
print(Ybus)

   From_line  To_line      R      X  Full_line_B
0          1        2  0.050  0.250       0.0500
1          2        3  0.025  0.125       0.1000
2          1        4  0.020  0.200       0.0333
3          2        4  0.020  0.200       0.0333
4          4        5  0.010  0.100       0.0200
[[ 1.26428027 -8.7549989j  -0.76923077 +3.84615385j
   0.         +0.j         -0.4950495  +4.95049505j
   0.         +0.j        ]
 [-0.76923077 +3.84615385j  2.80274181-16.39730659j
  -1.53846154 +7.69230769j -0.4950495  +4.95049505j
   0.         +0.j        ]
 [ 0.         +0.j         -1.53846154 +7.69230769j
   1.53846154 -7.64230769j  0.         +0.j
   0.         +0.j        ]
 [-0.4950495  +4.95049505j -0.4950495  +4.95049505j
   0.         +0.j          1.98019802-19.7586802j
  -0.99009901 +9.9009901j ]
 [ 0.         +0.j          0.         +0.j
   0.         +0.j         -0.99009901 +9.9009901j
   0.99009901 -9.8909901j ]]


In [278]:
#Function to caluculate the known P values
def P_Calc(V, YBus, BusNum, delta, P):
    P_Calc = np.zeros(BusNum, dtype=float)
    for i in range (BusNum):
        if np.isnan(P[i]):
            P_Calc[i] == 0
        else:
            for j in range (BusNum):
                P_Calc[i] += (V[i]*V[j]*(np.real(YBus)[i][j]*cmath.cos(delta[i]-delta[j])
                            +np.imag(YBus)[i][j]*cmath.sin(delta[i]-delta[j]))) 
    return P_Calc


#Function to caluculate the known Q values
def Q_Calc(V, YBus, BusNum, delta, Q):
    Q_Calc = np.zeros(BusNum, dtype=float)
    for i in range (BusNum):
        if np.isnan(Q[i]):
            Q_Calc[i] == 0
        else:
           for j in range (BusNum):
                Q_Calc[i] += (V[i]*V[j]*(np.real(YBus)[i][j]*cmath.sin(delta[i]-delta[j])
                                         -np.imag(YBus)[i][j]*cmath.cos(delta[i]-delta[j]))) 
    return Q_Calc



#Initial guesses
num_buses = 5
for x in range(num_buses):
    if(np.isnan(V[x])):
        V[x] = 1
    if(np.isnan(delta[x])):
        delta[x] = 0

P_calc = (P_Calc(V, Ybus, num_buses, delta, P))
Q_calc = (Q_Calc(V, Ybus, num_buses, delta, Q))

print(P_calc)
print(Q_calc)

[5.55111512e-17 5.55111512e-17 0.00000000e+00 0.00000000e+00
 0.00000000e+00]
[ 0.      -0.09165  0.      -0.0433  -0.01   ]


C:\Users\asbjo\AppData\Local\Temp\ipykernel_11348\243242556.py:9: ComplexWarning: Casting complex values to real discards the imaginary part
  P_Calc[i] += (V[i]*V[j]*(np.real(YBus)[i][j]*cmath.cos(delta[i]-delta[j])
C:\Users\asbjo\AppData\Local\Temp\ipykernel_11348\243242556.py:22: ComplexWarning: Casting complex values to real discards the imaginary part
  Q_Calc[i] += (V[i]*V[j]*(np.real(YBus)[i][j]*cmath.sin(delta[i]-delta[j])


In [281]:
#Jacobian
V, delta, P, Q, bus_type = buses('Busdata.csv')

class PQ:
    def __init__(self, Bus_type, Bus_num, vec_pos):
        self.Bus_type = Bus_type
        self.Bus_num = Bus_num
        self.vec_pos = vec_pos

class VT:
    def __init__(self, Bus_type, Bus_num, vec_pos):
        self.Bus_type = Bus_type
        self.Bus_num = Bus_num
        self.vec_pos = vec_pos


def jacobian_init(P,Q, V, delta):

    #Endre hardkoding av 7 her
    PQ_vec =  np.zeros(7, dtype=float)
    PQ_jacobian = []
    VT_jacobian = []
    numP = 0
    numQ = 0
    rowP = 0
    rowQ = 0
    for x in range(len(P)):
        
        if (np.isnan(P[x]) == False):
            PQ_vec[numP] = P[x]
            businfo = PQ("P", rowP, numP)
            PQ_jacobian.append(businfo)
            numP += 1
        rowP += 1
    for x in range(len(Q)):
        if (np.isnan(Q[x]) == False):
            PQ_vec[numP + numQ] = Q[x]
            businfo = PQ("Q", rowQ, numQ)
            PQ_jacobian.append(businfo)
            numQ += 1
        rowQ += 1

    numV = 0
    numT = 0
    rowV = 0
    rowT = 0
    for x in range(len(delta)):
        if (np.isnan(delta[x])):
            businfo = VT("T", rowT, numT)
            VT_jacobian.append(businfo)
            numT += 1
        rowT += 1    

    for x in range(len(V)):    
        if (np.isnan(V[x])):
            businfo = VT("V", rowV, numV)
            VT_jacobian.append(businfo)
            numV += 1
        rowV += 1

    return VT_jacobian, PQ_jacobian, PQ_vec


VT_jacobian, PQ_jacobian, PQ_vec = jacobian_init(P, Q, V, delta)
 


def make_jacobian(VT_jacobian, PQ_jacobian, PQ_vec, num_buses, V, delta):
    j = np.zeros((7,7), dtype=complex)

    V = [1,1,1,1,1]
    delta = [0,0,0,0,0]
    
    for x in range(len(PQ_vec)):
        for y in range(len(PQ_vec)):
            if (PQ_jacobian[x].Bus_num == VT_jacobian[y].Bus_num):
                if (PQ_jacobian[x].Bus_type == 'P' and VT_jacobian[y].Bus_type == 'T'):
                    for i in range(num_buses):
                        if (i==PQ_jacobian[x].Bus_num):
                            j[x,y] += 0
                        else:
                            j[x,y] += V[PQ_jacobian[x].Bus_num]*(-np.real(Ybus[PQ_jacobian[x].Bus_num,i])*cmath.sin(delta[PQ_jacobian[x].Bus_num]-delta[i])+np.imag(Ybus[PQ_jacobian[x].Bus_num,i])*1j*cmath.cos(delta[PQ_jacobian[x].Bus_num]-delta[i]))      

                if (PQ_jacobian[x].Bus_type == 'P' and VT_jacobian[y].Bus_type == 'V'):
                    for i in range(num_buses):
                        if (i==PQ_jacobian[x].Bus_num):
                            j[x,y] += 2*V[PQ_jacobian[x].Bus_num]*np.real(Ybus[PQ_jacobian[x].Bus_num,i])
                        else:
                            j[x,y] += V[PQ_jacobian[x].Bus_num]*(np.real(Ybus[PQ_jacobian[x].Bus_num,i])*cmath.cos(delta[PQ_jacobian[x].Bus_num]-delta[i])+np.imag(Ybus[PQ_jacobian[x].Bus_num,i])*1j*cmath.sin(delta[PQ_jacobian[x].Bus_num]-delta[i]))      

                if (PQ_jacobian[x].Bus_type == 'Q' and VT_jacobian[y].Bus_type == 'T'):
                    j[x,y] += V[PQ_jacobian[x].Bus_num]*V[y]*(np.real(Ybus[PQ_jacobian[x].Bus_num,i]*cmath.cos(delta[PQ_jacobian[x].Bus_num]-delta[i]))+np.imag(Ybus[PQ_jacobian[x].Bus_num,i]*cmath.sin(delta[PQ_jacobian[x].Bus_num]-delta[i]))*1j)

                if (PQ_jacobian[x].Bus_type == 'Q' and VT_jacobian[y].Bus_type == 'V'):
                    j[x,y] += 2*V[PQ_jacobian[x].Bus_num]*(np.real(Ybus[PQ_jacobian[x].Bus_num,i])*cmath.sin(delta[PQ_jacobian[x].Bus_num]-delta[i])-np.imag(Ybus[PQ_jacobian[x].Bus_num,i])*1j*cmath.cos(delta[PQ_jacobian[x].Bus_num]-delta[i]))
           
            else:
                if (PQ_jacobian[x].Bus_type == 'P' and VT_jacobian[y].Bus_type == 'T'):
                    j[x,y] += V[PQ_jacobian[x].Bus_num]*V[VT_jacobian[y].Bus_num]*(np.real(Ybus[PQ_jacobian[x].Bus_num,VT_jacobian[y].Bus_num])*cmath.sin(delta[PQ_jacobian[x].Bus_num]-delta[VT_jacobian[y].Bus_num])-np.imag(Ybus[PQ_jacobian[x].Bus_num,VT_jacobian[y].Bus_num])*1j*cmath.cos(delta[PQ_jacobian[x].Bus_num]-delta[VT_jacobian[y].Bus_num]))
                
                if (PQ_jacobian[x].Bus_type == 'P' and VT_jacobian[y].Bus_type == 'V'):
                    j[x,y] += V[PQ_jacobian[x].Bus_num]*(np.real(Ybus[PQ_jacobian[x].Bus_num,VT_jacobian[y].Bus_num])*cmath.cos(delta[PQ_jacobian[x].Bus_num]-delta[VT_jacobian[y].Bus_num])+np.imag(Ybus[PQ_jacobian[x].Bus_num,VT_jacobian[y].Bus_num])*1j*cmath.sin(delta[PQ_jacobian[x].Bus_num]-delta[VT_jacobian[y].Bus_num]))
                
                if (PQ_jacobian[x].Bus_type == 'Q' and VT_jacobian[y].Bus_type == 'V'):
                    j[x,y] += V[PQ_jacobian[x].Bus_num]*np.real(Ybus[PQ_jacobian[x].Bus_num,VT_jacobian[y].Bus_num]*cmath.sin(delta[PQ_jacobian[x].Bus_num]-delta[VT_jacobian[y].Bus_num])-np.imag(Ybus[PQ_jacobian[x].Bus_num,VT_jacobian[y].Bus_num]*cmath.cos(delta[PQ_jacobian[x].Bus_num]-delta[VT_jacobian[y].Bus_num]))*1j)
                
                if (PQ_jacobian[x].Bus_type == 'Q' and VT_jacobian[y].Bus_type == 'T'):
                    j[x,y] += V[PQ_jacobian[x].Bus_num]*V[VT_jacobian[y].Bus_num]*(np.real(Ybus[PQ_jacobian[x].Bus_num,VT_jacobian[y].Bus_num]*cmath.cos(delta[PQ_jacobian[x].Bus_num]-delta[VT_jacobian[y].Bus_num]))-np.imag(Ybus[PQ_jacobian[x].Bus_num,VT_jacobian[y].Bus_num]*cmath.sin(delta[PQ_jacobian[x].Bus_num]-delta[VT_jacobian[y].Bus_num]))*1j)
    return j


j = make_jacobian(VT_jacobian, PQ_jacobian, PQ_vec, num_buses, V_calc, delta_calc)

print(j)
    

[[ 0.         +8.7966489j   0.         -3.84615385j
   0.         -4.95049505j  0.         +0.j
  -0.76923077 +0.j         -0.4950495  +0.j
   0.         +0.j        ]
 [ 0.         -3.84615385j  0.        +16.48895659j
   0.         -4.95049505j  0.         +0.j
   2.80274181 +0.j         -0.4950495  +0.j
   0.         +0.j        ]
 [ 0.         -4.95049505j  0.         -4.95049505j
   0.        +19.8019802j   0.         -9.9009901j
  -0.4950495  +0.j          1.98019802 +0.j
  -0.99009901 +0.j        ]
 [ 0.         +0.j          0.         +0.j
   0.         -9.9009901j   0.         +9.9009901j
   0.         +0.j         -0.99009901 +0.j
   0.99009901 +0.j        ]
 [-0.76923077 +0.j          0.         +0.j
  -0.4950495  +0.j          0.         +0.j
   0.         +0.j          0.         +0.j
   0.         +0.j        ]
 [-0.4950495  +0.j         -0.4950495  +0.j
  -0.99009901 +0.j         -0.99009901 +0.j
   0.         +0.j          0.        -19.8019802j
   0.         +0.j     

In [280]:
# Inverse of jacobian

j_inv = np.linalg.inv(j)

print(j_inv)

[[ 0.00000000e+00-8.99503586e-02j  0.00000000e+00-2.99180829e-02j
   0.00000000e+00-2.96133582e-02j  0.00000000e+00-2.97649618e-02j
  -6.12511743e-01-0.00000000e+00j  1.52362330e-03+0.00000000e+00j
   7.58784103e-06+0.00000000e+00j]
 [ 0.00000000e+00+8.82561245e-01j  0.00000000e+00+3.81187638e-01j
   0.00000000e+00+7.86744078e-01j  0.00000000e+00+7.85969734e-01j
  -3.12852152e+00-0.00000000e+00j  7.78219902e-03+0.00000000e+00j
   3.87563572e-05+0.00000000e+00j]
 [ 0.00000000e+00+1.39769019e-01j  0.00000000e+00+4.64880980e-02j
   0.00000000e+00+4.60146028e-02j  0.00000000e+00+4.62501714e-02j
  -1.06825098e+00-0.00000000e+00j -2.36747621e-03-0.00000000e+00j
  -1.17903376e-05-0.00000000e+00j]
 [ 0.00000000e+00+1.43148219e-01j  0.00000000e+00+4.78311461e-02j
   0.00000000e+00+4.83675637e-02j  0.00000000e+00-5.23963163e-02j
  -1.08828598e+00-0.00000000e+00j  2.68208806e-03+0.00000000e+00j
   5.04328772e-03+0.00000000e+00j]
 [ 5.06280623e+00+0.00000000e+00j  2.55593819e+00+0.00000000e+00j
  